<a href="https://colab.research.google.com/github/zahraDehghanian97/AI_booklet_CE-AUT/blob/master/ddcrp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 23.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910476 sha256=2932a3c4e877f09f28f36c88e65914cb43b5d390d91abbe6cd0649c229f769f5
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [6]:
from torch_geometric.datasets import Planetoid
import numpy as np
from numpy import seterr, isneginf, array
from scipy.special import gammaln
from scipy.special import logsumexp


dataset = Planetoid("/content/sample_data/Cora" , name = 'Cora')

Processing...
Done!


In [35]:
print(dataset.data.keys)
features = dataset.data.x
x = features.detach().cpu().numpy()[0:100]
print(x.shape)

# x = np.array([[10,0,0,0,0],[10,0,0,0,0],[5,0,0,0,0],[11,0,0,0,1],[0,10,0,0,1],[0,10,0,0,0],[0,0,10,0,0],[0,1,10,0,0],[20,0,2,0,0],[10,0,0,1,0],[10,1,0,10,0],[10,0,2,10,0],[10,0,0,10,0],[10,1,0,1,0],[10,0,0,0,0],])


['x', 'edge_index', 'train_mask', 'test_mask', 'val_mask', 'y']
(100, 1433)


/usr/local/lib/python3.10/dist-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [43]:
#bookkeeper data
#cluster,link,obs,lhood,
def dirichlet_likelihood(Xp, hyper):
    if len(Xp.shape) == 2:
        X =sum(Xp)
    else:
        X = Xp
    idx = np.where(X!=0)
    lh = gammaln(len(X)*hyper) + sum(gammaln(X[idx]+hyper))\
    -len(idx)*gammaln(hyper)  - gammaln(sum(X)+len(X) * hyper)
    return lh



hyper = 0.001 #0.2
lhood_fn = lambda x:dirichlet_likelihood(x,hyper)


In [44]:
def window_delay(a,size=1):
    if abs(a) <= size and a >= 0:
        return 1;
    else:
        return 0;



window_size = 20 #20
delay = lambda x:window_delay(x,window_size)


In [45]:
def linear_distance(i,j):
    return i-j


#get the customers linked to customer i
def get_linked(i,link):
    c = []
    q = []
    q.append(i)
    while q:
        cur = q[0]
        c.append(cur)
        for k in range(0,len(link)):
            if (link[k] == cur) and (k not in c) and (k not in q):
                q.append(k)
        q = q[1:]
    return c

def ddcrp_infer(obs,lhood_fn,distance,delay,n_iter,alpha = 0.2):
    n = len(obs)
    cluster = np.array([0]*n)
    link = np.array([0]*n)
    prior = np.random.random(n*n).reshape((n,n))
    #lhood = np.random.random(n)
    merged_lhood = np.random.random(n)
    lhood = list(map(lambda x: lhood_fn(obs[np.where(cluster == x)]) , cluster))  #lhood of each cluster

    obs_lhood = 0 #the likelihood of all obs

    #prior of each customer
    for i in range(0,n):
        for j in range(0,n):
            try:
                if i==j:
                    prior[i][j] = np.log(alpha)
                else:
                    seterr(divide='ignore')
                    prior[i][j] = np.log(delay(distance(i,j)))
                    seterr(divide='warn')
                    prior[i][j][isneginf(prior[i][j])] = 0
            except Exception as e:
                # print(e)
                pass


    for t in range(0,n_iter):
        print("iter "+str(t))
        obs_lhood = 0
        for i in range(0,n):
            #print ("sample"+str(i)+"th:")
            #remove the ith's link
            old_link = link[i]
            old_cluster = cluster[old_link]
            cluster[i] = i
            link[i] = i
            linked = get_linked(i,link)
            # print(linked)
            cluster[linked] = i

            if old_cluster not in linked :
                idx = np.where(cluster == old_cluster)
                lhood[old_cluster] = lhood_fn(obs[idx])
                lhood[i] = lhood_fn(obs[linked])


            #calculate the likelihood of the merged cluster
            for j in np.unique(cluster):

                if j == cluster[i] :
                    merged_lhood[j] = 2*lhood_fn(obs[linked])
                else:
                    merged_lhood[j] = lhood_fn(np.concatenate((obs[linked] , obs[np.where(cluster == j)])))

            log_prob = list(map(lambda x: prior[i][x] + merged_lhood[cluster[x]] - lhood[cluster[x]]-lhood[cluster[i]], np.arange(n)))
            prob = np.exp(log_prob - logsumexp(log_prob))

            #sample z_i
            link[i] = np.random.choice(np.arange(n),1,p=prob)

            #update the likelihood if the link sample merge two cluster
            new_cluster = cluster[link[i]]
            if new_cluster !=i:
                cluster[linked] = new_cluster
                lhood[new_cluster] = merged_lhood[new_cluster]

            #cal the likelihood of all obs
            for u in np.unique(cluster):
                obs_lhood = obs_lhood + lhood[u]

        print("cluster")
        print(cluster)
        print("link")
        print(link)
        print(obs_lhood)

    return cluster,link,obs_lhood

In [46]:
#initalize parameters
n_iter =20 #200
alpha = 0.0000000000000001  #0.2

distance = linear_distance
cluster,link,lhood = ddcrp_infer(x,lhood_fn,distance,delay,n_iter,alpha)

iter 0
cluster
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
link
[ 0  0  0  1  1  2  1  3  5  4  2  5  6  7 11]
-3473.440906399421
iter 1
cluster
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
link
[0 0 1 0 0 3 0 4 3 3 8 4 0 3 3]
-3473.440906399421
iter 2
cluster
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
link
[0 0 1 0 1 1 3 1 5 5 8 3 3 4 7]
-3473.440906399421
iter 3
cluster
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
link
[ 0  0  1  2  0  2  1  2  6  5  4  4  2  4 10]
-3473.440906399421
iter 4
cluster
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
link
[ 0  0  1  1  1  1  0  4  5  7  1  1  5  0 13]
-3473.440906399421
iter 5
cluster
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
link
[ 0  0  0  2  2  3  0  0  4  8  3 10  8  9  1]
-3473.440906399421
iter 6
cluster
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
link
[0 0 1 2 1 3 5 5 6 2 4 2 8 8 4]
-3473.440906399421
iter 7
cluster
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
link
[0 0 1 1 1 2 4 0 7 2 4 2 0 5 0]
-3473.440906399421
iter 8
cluster
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
link
[ 0  0  0  0  1  4  5  1  6  0  4  8  1 10 10]
-3473.44090639942